In [4]:
%pip install rx

  Using cached Rx-3.2.0-py3-none-any.whl.metadata (4.6 kB)
Using cached Rx-3.2.0-py3-none-any.whl (199 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from rx import create

def generador(observer, schedule):
    observer.on_next("Hola")

    #Cuando se detecta el mensaje de erorr, ya no se continua con la ejecucion
    #observer.on_error("Error")
    observer.on_next("Mundo")

    #No lleva parametros; notificas y dices que la funcion ha terminado
    observer.on_completed()


observable = create(generador)

observable.subscribe(on_next = lambda s: print(f"Recibido: {s}"),
                     on_error = lambda e: print(f"Mensaje de Error: {e}"),
                     on_completed = lambda: print("Finalizado"))

observable.subscribe(on_next = lambda s: print(f"Recibido2: {s}"),
                     on_error = lambda e: print(f"Mensaje de Error2: {e}"),
                     on_completed = lambda: print("Finalizado2"))

Recibido: Hola
Recibido: Mundo
Recibido2: Hola
Recibido2: Mundo


In [13]:
from rx import of

observable = of('Hola', 'Adios', 'Mundo')

observable.subscribe(on_next = lambda s: print(f"Recibido: {s}"),
                     on_error = lambda e: print(f"Mensaje de Error: {e}"),
                     on_completed = lambda: print("Finalizado"))


Recibido: Hola
Recibido: Adios
Recibido: Mundo
Finalizado


In [15]:
from rx import from_

observable = from_({1, 2, 3, 4, 5})

observable.subscribe(on_next = lambda s: print(f"Recibido: {s}"),
                     on_error = lambda e: print(f"Mensaje de Error: {e}"),
                     on_completed = lambda: print("Finalizado"))

Recibido: 1
Recibido: 2
Recibido: 3
Recibido: 4
Recibido: 5
Finalizado


Podemos crear observers especializados heredando de la clase Observer; hacer que se escuche de teclado

In [17]:
from rx import create
from rx.core import Observer

#La clase printer, hereda de observer -----------------------------------------------------------------
class Printer(Observer):
    #Sobreescribimos la funcion de "on_next"
    def on_next(self, value):
        print(f"Recibido: {value}")

    def on_error(self, error):
        print(f"Mensaje de Error: {error}")

    def on_completed(self):
        print("Finalizado")

#FUNCION OBSERVADORA, LA QUE VA A MIRAR LO QUE SE PULSE ---------------------------------------------
def observer_teclado(observer, schedule):
    exit_ = False
    while not exit_:
        msg = input("Introduce un mensaje: ")
        if msg:
            if msg == "exit":
                exit_ = True
                observer.on_completed()
            else:
                observer.on_next(msg)
        else:
            observer.on_error("Error! No ha habido mensaje!")
            exit_ = True

observable = create(observer_teclado)

#para distinguirlos...
printer = Printer()

observable.subscribe(printer)

Recibido: hola
Finalizado


In [20]:
from rx import create, operators
from rx.core import Observer

#La clase printer, hereda de observer -----------------------------------------------------------------
class Printer(Observer):
    #Sobreescribimos la funcion de "on_next"
    def on_next(self, value):
        print(f"Recibido: {value}")

    def on_error(self, error):
        print(f"Mensaje de Error: {error}")

    def on_completed(self):
        print("Finalizado")

#FUNCION OBSERVADORA, LA QUE VA A MIRAR LO QUE SE PULSE ---------------------------------------------
def observer_teclado(observer, schedule):
    exit_ = False
    while not exit_:
        msg = input("Introduce un mensaje: ")
        if msg:
            if msg == "exit":
                exit_ = True
                observer.on_completed()
            else:
                observer.on_next(msg)
        else:
            observer.on_error("Error! No ha habido mensaje!")
            exit_ = True

observable = create(observer_teclado)

#Solamente funciona con los datos que llegan a traves de "on_next"; porque lo que le llegue por "on_error", no son datos 
observable_intermedio = observable.pipe(operators.map(lambda s: s.upper()), 
                                        operators.map(lambda s: f"Convertido a mayusculas {s}"))

#para distinguirlos...
printer = Printer()

observable_intermedio.subscribe(printer)

Recibido: Convertido a mayusculas HOLA
Finalizado


In [23]:
observable = from_([456, 20, 123, 56, 89, 100, 9648, 213])

#observable.subscribe(on_next = lambda s: print(s), on_completed=lambda: print("Finalizado"), on_error=lambda e: print(f"Error: {e}"))

#observable.pipe(operators.max()).subscribe(on_next = lambda s: print(s), on_completed=lambda: print("Finalizado"), on_error=lambda e: print(f"Error: {e}"))


observable.pipe(operators.filter(lambda s: s > 100)).subscribe(on_next = lambda s: print(s), on_completed=lambda: print("Finalizado"), on_error=lambda e: print(f"Error: {e}"))

456
123
9648
213
Finalizado


In [ ]:
from rx import create, operators
import time

def nun_generator(observer, scheduler):
    l = [456, 20, 123, 56, 89, 100, 9648, 213]
    for n in l:
        observer.on_next(n)
        time.sleep(1)
    observer.on_completed()

observable = create(nun_generator)

observable.pipe(operators.filter(lambda s: s > 100)).subscribe(on_next = lambda s: print(s), on_completed=lambda: print("Finalizado"), on_error=lambda e: print(f"Error: {e}"))


456
123
9648
213
Finalizado


Ejercicio OMDB API

In [10]:
from rx import create, operators
from requests import get
from rx.core import Observer

api_key = "fbd46cff"

class Printer(Observer):
    #Sobreescribimos la funcion de "on_next"
    def on_next(self, value):
        print(f"Recibido: {value}")

    def on_error(self, error):
        print(f"Mensaje de Error: {error}")

    def on_completed(self):
        print("Finalizado")

def search_movies(observer, scheduler):
    while 1:
        movie = input("Introduce una pelicula: ")
        response = get(f"http://www.omdbapi.com/?apikey={api_key}&s={movie}")

        if response.status_code == 200:
            for i in response.json()["Search"]:
                observer.on_next(i)
        else:
            observer.on_error("Error en la peticion")

observable = create(search_movies)

observable.pipe(operators.filter(lambda d: d["Type"] == "movie"),
                operators.map(lambda d: f"({d['imdbID']}) - {d['Title']} : {d['Poster']} ({d['Year']})")).subscribe(Printer())



Recibido: (tt0317219) - Cars : https://m.media-amazon.com/images/M/MV5BMTg5NzY0MzA2MV5BMl5BanBnXkFtZTYwNDc3NTc2._V1_SX300.jpg (2006)
Recibido: (tt1216475) - Cars 2 : https://m.media-amazon.com/images/M/MV5BMTUzNTc3MTU3M15BMl5BanBnXkFtZTcwMzIxNTc3NA@@._V1_SX300.jpg (2011)
Recibido: (tt3606752) - Cars 3 : https://m.media-amazon.com/images/M/MV5BMTc0NzU2OTYyN15BMl5BanBnXkFtZTgwMTkwOTg2MTI@._V1_SX300.jpg (2017)
Recibido: (tt0200027) - Riding in Cars with Boys : https://m.media-amazon.com/images/M/MV5BZmQ3MjM3OTAtNDUwYy00NWM3LTljM2ItN2JhZTRlMDM3ZTcxXkEyXkFqcGc@._V1_SX300.jpg (2001)
Recibido: (tt0081698) - Used Cars : https://m.media-amazon.com/images/M/MV5BNTY3Zjg2ODQtNjdhNS00OWY1LThkODEtYWYyNDliNzI3ZTlhXkEyXkFqcGc@._V1_SX300.jpg (1980)
Recibido: (tt1282139) - Cars of the Revolution : https://m.media-amazon.com/images/M/MV5BZmE3NGIzZmEtZTBhNi00ZDYzLWJmZjItZWRmMmYzYzVmNTViXkEyXkFqcGdeQXVyMjExNjgyMTc@._V1_SX300.jpg (2008)
Recibido: (tt0246692) - Old Men in New Cars : https://m.media-amazon.co

TKInter -- Para plataformas GUI en Python

In [15]:
from tkinter import Tk, Label

#Creamos la ventana
window = Tk()

window.geometry("200x200")

#Creamos la etiqueta; y tengo que decirle en que ventana se va a mostrar (window) y lueog el texto que quiero que muestre
label = Label(text="Hola Mundo")
label.pack()

window.mainloop()

Widgets

In [ ]:
from tkinter import Tk, Label, Button, Entry, Checkbutton, BooleanVar, StringVar
from tkinter.ttk import Combobox

class Window:

    def __init__(self):
        #Creamos la ventana
        self.window = Tk()

        #window.geometry("200x200")

        #Creamos la etiqueta; y tengo que decirle en que ventana se va a mostrar (window) y lueog el texto que quiero que muestre
        self.label = Label(text="Hola", font=("Arial Bold", 50))
        self.label.grid(row=0, column=0)

        # Creamos la etiqueta 2 -----------------------------------------------------
        self.label2Text = StringVar()
        self.label2Text.set("Valor nuevo")

        #Creamos la etiqueta; y tengo que decirle en que ventana se va a mostrar (window) y lueog el texto que quiero que muestre
        self.label2 = Label(font=("Arial", 36), textvariable=self.label2Text)
        self.label2.grid(row=0, column=1, padx=100)

        #Creamos el boton
        self.button = Button(text="Pulsa", command=self.read_entry)

        #Lo mostramos en la ventana
        self.button.grid(row=1, column=1)

        self.entry = Entry()
        self.entry.grid(row=1, column=0)

        #Creamos el combobox -----------------------------------------------------
        self.comboBox = Combobox(values=["cero", "uno", "dos", "tres"])
        self.comboBox.grid(row=2, column=0)

        #Si queremos uno por defecto
        self.comboBox.current(2)

        #cHECK BUTTON -----------------------------------------------------------
        self.checkButton = Checkbutton(text="Aceptar")
        self.checkButton.grid(row=3, column=0)

        self.checkButtonState = BooleanVar()
        self.checkButtonState.set(True)

        self.checkButton = Checkbutton(text="yes or no", variable=self.checkButtonState)
        self.checkButton.grid(row=4, column=0)

        self.window.mainloop()

    #Configure; es cuando hay widgets que se tienen que configurar

    def read_entry(self):
        self.checkButtonState.set(not self.checkButtonState.get())
        self.label2Text.set(self.entry.get())

Window()